## Duckietown Import

In [2]:
import os 
if not os.path.isdir('Self-Driving-in-Duckietown') and not os.path.isdir('../Self-Driving-in-Duckietown'):
    branch = "milestone_3"
    !git clone --branch {branch} https://github.com/dancsomarci/Self-Driving-in-Duckietown
    !pip3 install -e gym-duckietown

In [3]:
if "/Self-Driving-in-Duckietown" not in os.getcwd():
    os.chdir('Self-Driving-in-Duckietown')

In [4]:
!pip3 install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/Self-Driving-in-Duckietown
  Running setup.py develop for duckietown-gym-daffy
ERROR: Command errored out with exit status 1: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/content/Self-Driving-in-Duckietown/setup.py'"'"'; __file__='"'"'/content/Self-Driving-in-Duckietown/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' develop --no-deps Check the logs for full command output.


## Initialize environment

In [5]:
!pip install stable_baselines3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import gym
import numpy as np
import torch as th
import matplotlib.pyplot as plt
import gym_duckietown
from env import launch_env


from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback

DEBUG:commons:version: 6.2.4 *
DEBUG:typing:version: 6.2.3
DEBUG:duckietown_world:duckietown-world version 6.4.3 path /usr/local/lib/python3.8/dist-packages
DEBUG:geometry:PyGeometry-z6 version 2.1.4 path /usr/local/lib/python3.8/dist-packages
DEBUG:aido_schemas:aido-protocols version 6.1.1 path /usr/local/lib/python3.8/dist-packages
DEBUG:nodes:version 6.2.17 path /usr/local/lib/python3.8/dist-packages pyparsing 3.0.9
DEBUG:gym-duckietown:gym-duckietown version 6.1.31 path /content/Self-Driving-in-Duckietown



{'audio': ('xaudio2', 'directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_gl_shaders': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_input': False, 'debug_x11': False, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'search_local_libs': True, 'win32_gdi_font': False, 'headless': False, 'headless_device': 0, 'win32_disable_shaping': False}


In [7]:
env = launch_env()

/usr/local/lib/python3.8/dist-packages/pyglet/canvas/headless.py:65: UserWarning: No device available for EGL device platform. Using native display type.
  warnings.warn('No device available for EGL device platform. Using native display type.')


ValueError: ignored

In [ ]:
%load_ext tensorboard
tb_log = "..\\log\\" #TODO

In [ ]:
ppo_model = PPO(policy='MlpPolicy',
                env=env,
                learning_rate=5.e-5,
                gamma=0.99,
                tensorboard_log=tb_log,
                verbose=1)

## Learning

In [ ]:
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path="..\\saves\\", #TODO
  name_prefix="ppo_model",)

In [ ]:
ppo_model.learn(total_timesteps=100000, callback=checkpoint_callback)

## Model Loading

In [ ]:
env = launch_env(map="straight_road")
ppo_model = PPO.load("..\\saves\\ppo_model_80000_steps")

## Model Logging

In [ ]:
%tensorboard --logdir tb_log

## Model Testing

In [ ]:
cutoff = 256
with th.no_grad():
    while True:
        obs = env.reset()
        env.render()
        rewards = []
        steps = 0
        while True:
            action, _states = ppo_model.predict(obs, deterministic=True)
            obs, rew, done, misc = env.step(action)
            rewards.append(rew)
            env.render()
            steps += 1
            if done or steps >= cutoff:
                break
        print("mean episode reward:", np.mean(rewards))